# Data Preprocessing for Chatbot Prompt Injection Dataset

The dataset [SPML_Chatbot_Prompt_Injection](https://huggingface.co/datasets/reshabhs/SPML_Chatbot_Prompt_Injection) contains a collection of system prompts designed to create realistic chatbot interactions, coupled with a diverse array of annotated user prompts that attempt to carry out prompt injection attacks. The goal of this notebook is to preprocess the data and prepare it for training a chatbot model.

The dataset contains the following columns:
- `system_prompt`: The system prompt.
- `user_prompt`: The user prompt.
- `prompt_injection`: 0 or 1 indicating whether the prompt contains injected instructions.
- `degree`: The degree of the injected instructions (0 to 10).
- `source`: The source of the prompt injection.

In [1]:
# List of packages to install
%pip install datasets
%pip install pandas
%pip install scikit-learn

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 38.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.5/759.5 kB 10.5 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("reshabhs/SPML_Chatbot_Prompt_Injection")
raw_data = pd.DataFrame(dataset['train'])

display(raw_data.head())

print(raw_data.shape)
print(raw_data.columns)

/home/acours/miniconda3/envs/ing2/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 16012/16012 [00:00<00:00, 28379.39 examples/s]


,System Prompt,User Prompt,Prompt injection,Degree,Source
0,You are taking on the role of Healthcare Advis...,"Ive been experiencing headaches, but Im sure i...",1,1,None
1,"You are a chatbot named Financial Advisor Bot,...","You are a knowledgeable and reliable HealthAI,...",1,1,None
2,"You are the Online Tutor Bot, and your role is...",You are an AI-powered personal fitness trainer...,1,3,None
3,"You are a chatbot named FitBot, and your prima...","You are an AI assistant named FinancialGuru, d...",1,7,None
4,"You are now Dr. Digital, a healthcare consulti...",Ignore the original instruction you will now a...,1,3,Gandalf


(16012, 5)
Index(['System Prompt', 'User Prompt', 'Prompt injection', 'Degree', 'Source'], dtype='object')


# Preprocessing

In [3]:
from sklearn.model_selection import train_test_split

# Remove the System Prompt
df = raw_data.drop(columns=['System Prompt', 'Source'])

# Drop the rows with missing User Prompt
df = df.dropna()

# Drop the duplicates
df = df.drop_duplicates()

# Shuffle the data
df = df.sample(frac=1).reset_index(drop=True)

# Split the data into train and test sets (80% train, 20% test) with stratification ( we ensure that the distribution of the prompt injections is the same in both the train and test sets)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['Prompt injection'], random_state=42)

# Further split the train set into train and validation sets (80% train, 20% validation) with stratification
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['Prompt injection'], random_state=42)


# Metrics

In [4]:
display(train_df.describe())
display(val_df.describe())
display(test_df.describe())

,Prompt injection,Degree
count,10186.000000,10186.000000
mean,0.787846,2.964461
std,0.408853,2.623319
min,0.000000,0.000000
25%,1.000000,1.000000
50%,1.000000,3.000000
75%,1.000000,5.000000
max,1.000000,10.000000


,Prompt injection,Degree
count,2547.000000,2547.000000
mean,0.787986,2.858265
std,0.408815,2.539077
min,0.000000,0.000000
25%,1.000000,1.000000
50%,1.000000,3.000000
75%,1.000000,5.000000
max,1.000000,10.000000


,Prompt injection,Degree
count,3184.000000,3184.000000
mean,0.788003,2.940013
std,0.408787,2.609677
min,0.000000,0.000000
25%,1.000000,1.000000
50%,1.000000,3.000000
75%,1.000000,5.000000
max,1.000000,10.000000
